_한국어로 기계번역됨_


# 서브그래프에 스레드 수준 지속성을 추가하는 방법


<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 가이드는 다음에 대한 기본 지식이 있음을 전제로 합니다:
        <ul>
            <li>            
                <a href="https://langchain-ai.github.io/langgraph/concepts/low_level/#subgraphs">
                    서브그래프
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/persistence/">
                    지속성
                </a>
            </li>
        </ul>
    </p>
</div>

이 가이드는 [서브그래프](https://langchain-ai.github.io/langgraph/how-tos/subgraph/)를 사용하는 그래프에 [스레드 수준](https://langchain-ai.github.io/langgraph/how-tos/persistence/)의 지속성을 추가하는 방법을 보여줍니다.


## 설정

먼저, 필요한 패키지를 설치합시다.


In [1]:
%%capture --no-stderr
%pip install -U langgraph


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 발견하고 성능을 향상시키세요. LangSmith는 trace 데이터를 사용하여 LangGraph로 구축된 LLM 앱을 디버그하고 테스트하며 모니터링할 수 있게 해줍니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 참조하세요.
    </p>
</div>


## 지속성 그래프 정의


서브그래프가 있는 그래프에 지속성을 추가하려면, **부모 그래프를 컴파일할 때** [체크포인터](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.base.BaseCheckpointSaver)를 전달하기만 하면 됩니다. LangGraph는 체크포인터를 자동으로 자식 서브그래프에 전파합니다.


!!! 주의
    서브그래프를 컴파일할 때 체크포인터를 제공해서는 안 됩니다. 대신, `parent_graph.compile()`에 전달할 **단일** 체크포인터를 정의해야 하며, LangGraph는 자동으로 체크포인터를 자식 서브그래프로 전파합니다. 체크포인터를 `subgraph.compile()`에 전달하면 무시됩니다. 이는 [서브그래프를 호출하는 노드 함수를 추가할 때](../subgraph#add-a-node-function-that-invokes-the-subgraph)에도 적용됩니다.


하나의 서브그래프 노드를 가진 간단한 그래프를 정의하여 이를 보여줍시다.


In [2]:
from langgraph.graph import START, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict


# subgraph


class SubgraphState(TypedDict):
    foo: str  # note that this key is shared with the parent graph state
    bar: str


def subgraph_node_1(state: SubgraphState):
    return {"bar": "bar"}


def subgraph_node_2(state: SubgraphState):
    # note that this node is using a state key ('bar') that is only available in the subgraph
    # and is sending update on the shared state key ('foo')
    return {"foo": state["foo"] + state["bar"]}


subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node_1)
subgraph_builder.add_node(subgraph_node_2)
subgraph_builder.add_edge(START, "subgraph_node_1")
subgraph_builder.add_edge("subgraph_node_1", "subgraph_node_2")
subgraph = subgraph_builder.compile()


# parent graph


class State(TypedDict):
    foo: str


def node_1(state: State):
    return {"foo": "hi! " + state["foo"]}


builder = StateGraph(State)
builder.add_node("node_1", node_1)
# note that we're adding the compiled subgraph as a node to the parent graph
builder.add_node("node_2", subgraph)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")


이제 메모리 체크포인터(`MemorySaver`)를 사용하여 그래프를 컴파일할 수 있습니다.


In [3]:
checkpointer = MemorySaver()
# You must only pass checkpointer when compiling the parent graph.
# LangGraph will automatically propagate the checkpointer to the child subgraphs.
graph = builder.compile(checkpointer=checkpointer)


## 지속성이 작동하는지 확인하기


이제 그래프를 실행하고 지속 상태를 검사하여 부모 그래프와 서브 그래프 모두에서 지속성이 작동하는지 확인해 봅시다. 우리는 `state.values`에서 부모 그래프와 서브 그래프 모두의 최종 실행 결과를 볼 수 있기를 기대합니다.


In [4]:
config = {"configurable": {"thread_id": "1"}}


In [5]:
for _, chunk in graph.stream({"foo": "foo"}, config, subgraphs=True):
    print(chunk)


{'node_1': {'foo': 'hi! foo'}}
{'subgraph_node_1': {'bar': 'bar'}}
{'subgraph_node_2': {'foo': 'hi! foobar'}}
{'node_2': {'foo': 'hi! foobar'}}


이제 그래프를 호출할 때 사용했던 것과 같은 설정으로 `graph.get_state()`를 호출하여 부모 그래프 상태를 볼 수 있습니다.


In [6]:
graph.get_state(config).values


{'foo': 'hi! foobar'}

서브그래프 상태를 보기 위해 두 가지 작업을 수행해야 합니다:

1. 서브그래프의 가장 최신 설정 값을 찾습니다.
2. `graph.get_state()`를 사용하여 가장 최신 서브그래프 설정에 대한 값을 가져옵니다.

올바른 설정을 찾기 위해, 부모 그래프의 상태 기록을 살펴보고 `node_2`(서브그래프가 있는 노드)에서 결과를 반환하기 전에 상태 스냅샷을 찾습니다:


In [7]:
state_with_subgraph = [
    s for s in graph.get_state_history(config) if s.next == ("node_2",)
][0]


상태 스냅샷은 다음에 실행할 `작업` 목록을 포함합니다. 서브그래프를 사용할 때 `작업`에는 서브그래프 상태를 검색하는 데 사용할 수 있는 구성이 포함됩니다:


In [8]:
subgraph_config = state_with_subgraph.tasks[0].state
subgraph_config


{'configurable': {'thread_id': '1',
  'checkpoint_ns': 'node_2:6ef111a6-f290-7376-0dfc-a4152307bc5b'}}

In [9]:
graph.get_state(subgraph_config).values


{'foo': 'hi! foobar', 'bar': 'bar'}

인간이 참여하는 작업 흐름을 위해 서브 그래프 상태를 수정하는 방법에 대해 더 알고 싶다면 이 [사용법 가이드](https://langchain-ai.github.io/langgraph/how-tos/subgraphs-manage-state/)를 확인해 보세요.
